In [ ]:
import java.io.File

val input = File("example.txt").readLines()
input

In [ ]:
val cardsToBid = input.map { it.split(" ").let { it[0] to it[1].toInt() } }
cardsToBid

In [ ]:
enum class HandType {
    HighCard,
    OnePair,
    TwoPair,
    ThreeOfAKind,
    FullHouse,
    FourOfAKind,
    FiveOfAKind,
}

data class Hand(
    val cards: String,
) : Comparable<Hand> {
    val handType: HandType
    val cardStrengths = "J23456789TQKA"

    init {
        val cardToCount = mutableMapOf<Char, Int>().withDefault { 0 }
        cards.forEach { cardToCount[it] = cardToCount.getValue(it) + 1 }
        val numberOfJokers = cardToCount.remove('J') ?: 0
        val cardWithHighestCount = cardToCount.maxByOrNull { it.value }?.key ?: cardStrengths.last()
        cardToCount[cardWithHighestCount] = cardToCount.getValue(cardWithHighestCount) + numberOfJokers

        handType = when {
            5 in cardToCount.values -> HandType.FiveOfAKind
            4 in cardToCount.values -> HandType.FourOfAKind
            3 in cardToCount.values && 2 in cardToCount.values -> HandType.FullHouse
            3 in cardToCount.values -> HandType.ThreeOfAKind
            cardToCount.values.count { it == 2 } == 2 -> HandType.TwoPair
            2 in cardToCount.values -> HandType.OnePair
            else -> HandType.HighCard
        }
    }


    override fun compareTo(other: Hand): Int {
        if (handType != other.handType) return compareValues(handType.ordinal, other.handType.ordinal)
        for ((card, otherCard) in cards.zip(other.cards)) {
            val comparison = compareValues(cardStrengths.indexOf(card), cardStrengths.indexOf(otherCard))
            if (comparison != 0) return comparison
        }
        return 0
    }

    override fun toString() = "$cards ($handType)"
}

%use dataframe
val handToBidFromWeakest = cardsToBid.map { (cards, bid) -> Hand(cards) to bid }.sortedBy { it.first }
handToBidFromWeakest.toDataFrame()

In [ ]:
handToBidFromWeakest.mapIndexed { index, pair -> (index + 1) * pair.second }.sum()